In [ ]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install tqdm

In [46]:
import spacy
import os
import csv
from tqdm import tqdm
import time

In [ ]:
import sys
!{sys.executable} -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [48]:
tokenizer = nlp.tokenizer
remove_words = ['nash', 'virtual']
def convert_text_to_doc(raw_text):
    text = raw_text.lower()
    for word in remove_words:
        text = text.replace(word, '')
    # tokenize text and remove stop words, out-of-vocab words, punctuation, and non alpha
    return nlp(" ".join([str(t) for t in tokenizer(text) if not t.is_stop and not t.is_oov and not t.is_punct and t.is_alpha]))

def get_family(source_doc):
    best_score = 0
    best_code = None
    best_family_name = None
    file = open('./family_codes.csv', "r")
    reader = csv.reader(file, delimiter=',')
    next(reader)
    # TODO: if score is not over a certain treshhold
    for family_code, family_name, include in reader:
        if bool(include):
            family_doc = convert_text_to_doc(family_name)
            score = family_doc.similarity(source_doc)
            if score > best_score:
                best_code = family_code
                best_family_name = family_name
                best_score = score
    return best_code, best_family_name, best_score

In [52]:
total_counter = 0
correct_counter_first = 0
correct_counter_second = 0
results = []
# cutoff = 10
cutoff = float('inf')
with open('./items_labeled_1.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    next(reader)
    for row in tqdm(reader):
        uuid, labeler, name, display_name, category, family_name_1, family_name_2 = row
        if bool(labeler) and bool(family_name_1):
            total_counter += 1
            text = f'{display_name if display_name else name} {category}'
            source_doc = convert_text_to_doc(text)
            best_code, best_family_name, best_score = get_family(source_doc)
            if best_family_name == family_name_1:
                correct_counter_first += 1
            elif best_family_name == family_name_2:
                correct_counter_second += 1
            results.append([text, source_doc, family_name_1, best_family_name, best_score])
            # print(f"NAME + CATEGORY: {source_doc}, EXPECTED: {family_name_1}, ACTUAL: {best_family_name}, SCORE: {best_score}")
            if total_counter == cutoff:
                break

first_correct = (correct_counter_first/total_counter)*100
second_correct = (correct_counter_second/total_counter)*100

fields = ['text', 'source_doc', 'expected', 'actual', 'similarity_score'] 
with open(f"./items_results_{len(results)}_{time.strftime('%H%M%S')}.csv", 'w') as new_file:
    csvwriter = csv.writer(new_file) 
    csvwriter.writerow(fields)
    csvwriter.writerows(results)
    csvwriter.writerow([f'Percentage correct on first: {(correct_counter_first/total_counter)*100}%'])
    csvwriter.writerow([f'Percentage correct on second: {(correct_counter_second/total_counter)*100}%'])
    csvwriter.writerow([f'Percentage incorrect: {((total_counter - correct_counter_first - correct_counter_second)/total_counter)*100}%'])


0it [00:00, ?it/s]/var/folders/s7/mgb89pzd4g530qcvhpdt5pkr0000gn/T/ipykernel_37208/1003730191.py:21: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = family_doc.similarity(source_doc)
1000it [15:52,  1.05it/s]
